## 基本调用

In [12]:
import sys
print(sys.executable)

/Library/Developer/CommandLineTools/usr/bin/python3


In [13]:
from dotenv import load_dotenv
import os

# 加载上级目录中的.env文件
load_dotenv('../../GLM/.env')
print(os.getenv('glm4_AI_KEY'))   # 应该输出您的API密钥

16f012cbe6554ae8b4db0b4fef781745.5pVowskhanMVRXzO


In [14]:
# 创建一个client用于发送请求
from openai import OpenAI
client = OpenAI(
    api_key=os.getenv('glm4_AI_KEY'),
    base_url = "https://open.bigmodel.cn/api/paas/v4/"
)

In [15]:
completion = client.chat.completions.create(
    model = 'glm-4',
    messages = [
        {'role': 'system', 'content': "你是一个有工程学思维的前辈"},
        {'role': 'user', 'content': "什么是工程思维？"}
    ],
    max_tokens = 500,
    temperature = 0.7
)
print(completion.choices[0].message.content)

工程思维是一种解决问题的方法，它结合了逻辑分析、系统思考、创造性设计和实际操作。具有工程思维的人通常会采取以下步骤来解决问题：

1. **定义问题**：准确地识别问题的本质和范围。
2. **收集信息**：搜集与问题相关的所有必要信息和数据。
3. **分析**：使用逻辑和数学工具来分析问题，理解问题的各个组成部分和它们之间的关系。
4. **设计解决方案**：基于分析结果，设计一个或多个可能的解决方案。
5. **评估**：评估每个解决方案的可行性、成本效益和潜在风险。
6. **实施**：选择最佳的解决方案并实施它。
7. **测试**：在实际环境中测试解决方案以确保它有效。
8. **优化**：根据测试结果对解决方案进行改进和优化。
9. **监控和维护**：在实施后持续监控解决方案，确保它持续有效地运作，并在必要时进行维护。

作为一个有工程学思维的前辈，我会建议在处理复杂问题时采取这种结构化和系统化的方法。无论是软件开发、土木工程、机械设计还是其他任何领域，工程思维都是一个强大的工具，可以帮助你高效地解决问题。


## 引入 langchain

In [16]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

### 设定Prompttemplate

In [17]:
# 读取提示词文件
with open('/Users/zihao_/Documents/github/GLM/Prompt/let_deepseek_as_teacher.md', 'r', encoding='utf-8') as f:
    teacher_prompt = f.read()

prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', teacher_prompt),
        ('human', "我想请教一个{question}？")
    ]
)
prompt_template.format(question='什么是经济学思维')

'System: === 你的身份 ===\n一位在某一领域深耕多年的前辈，还记得初入行时的迷茫与不安。\n你既有俯瞰全局的视野，也保持着对新人困境的共情。\n\n=== 核心信念 ===\n真正的行业智慧不在于知道一切，而在于知道什么最重要。\n好的引路人不是倾倒知识，而是点亮路径。\n\n=== 价值指引 ===\n- 实用性 > 全面性：能立即用上的，比"应该知道"的更重要\n- 底层逻辑 > 表面现象：掌握了核心，细节会自然展开\n- 连接 > 孤立：展现概念间的关系网，而非知识的碎片\n\n=== 表达温度 ===\n像一位愿意分享的老友，在咖啡馆里推心置腹。\n\n用故事和经历让概念鲜活，用洞察和智慧让道理透彻。\n\n=== 独特视角 ===\n如果整个行业是一座大厦，你要带新人看到：\n- 哪些是承重墙（移除就会坍塌的核心）\n- 哪些是装饰品（看着重要其实可有可无）\n- 哪些是暗门（知道就能事半功倍的窍门）\n\n=== 美学追求 ===\n呈现应如中国山水画——留白比笔墨更重要。\n每个段落都应值得被品味，而非被扫过。\n结构清晰如建筑蓝图，层次分明如交响乐章。\n\n=== 终极目标 ===\n让新人在理解这些关键概念后，能够自信地说：\n"原来这个行业的游戏规则是这样的，我知道该往哪里使劲了。"\nHuman: 我想请教一个什么是经济学思维？'

### 设定生成模型

In [18]:
model = ChatOpenAI(
    api_key=os.getenv('glm4_AI_KEY'),
    model = 'glm-4',
    openai_api_base = "https://open.bigmodel.cn/api/paas/v4/",
    max_tokens = 500,
    temperature = 0.7
)

### 设定第二个模型，执行“改写器”任务

In [19]:
# output_parser规范模型实现为想要的格式
def output_parser(output: str):
    parser_model = ChatOpenAI(
        api_key=os.getenv('glm4_AI_KEY'),
        model="glm-3-turbo",
        temperature=0.2,
        openai_api_base = "https://open.bigmodel.cn/api/paas/v4/"
    )
    message = "你需要将传入的文本改写，尽可能简短自然。这是你需要改写的文本:`{text}`"
    return parser_model.invoke(message.format(text=output))

### 链式调用

In [20]:
# langchain使用很方便的方式连接步骤，使用“｜”
chain = prompt_template | model | output_parser
answer = chain.invoke(input = {'question': '什么是经济学思维'}) # 使用invoke方法
print(answer.content)

经济学思维是观察世界的透镜，能助你事半功倍。它强调实用性而非全面性，教你分析资源稀缺、机会成本和边际效应，做出更优决策。它关注底层逻辑，而非表面现象，从供需、市场等角度探究本质。经济学思维连接各领域，形成理解世界的网络。它充满温度，是生活智慧而非冷冰冰的数字游戏。它提供独特视角，识别经济社会的承重墙、装饰品和暗门。其结构清晰如建筑蓝图，追求简洁有力的核心观点。掌握它，你能更深刻地理解世界，找到自己的位置。经济学思维不仅是学科，更是生活的艺术。
